In [3]:
import requests
from bs4 import BeautifulSoup

url = 'https://jobs.lever.co/ekimetrics/c62e4fd4-acd5-4860-857a-15a6797696be/apply'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Find all form inputs
form_inputs = soup.find_all('input')
for input_field in form_inputs:
    print(input_field.get('name'), input_field.get('type'))

resume file
name text
email email
phone text
location text
selectedLocation hidden
org text
cards[a6a97378-59b0-421f-a4c9-76737b41fee1][baseTemplate] hidden
cards[a6a97378-59b0-421f-a4c9-76737b41fee1][field0] radio
cards[a6a97378-59b0-421f-a4c9-76737b41fee1][field0] radio
cards[cc52355f-705b-40f9-9160-c9ff7fcbf769][baseTemplate] hidden
cards[cc52355f-705b-40f9-9160-c9ff7fcbf769][field0] radio
cards[cc52355f-705b-40f9-9160-c9ff7fcbf769][field0] radio
surveysResponses[fc802a09-b71e-40bd-a979-34722f5246c1][baseTemplate] hidden
surveysResponses[fc802a09-b71e-40bd-a979-34722f5246c1][surveyId] hidden
 hidden
surveysResponses[fc802a09-b71e-40bd-a979-34722f5246c1][responses][field0] checkbox
surveysResponses[fc802a09-b71e-40bd-a979-34722f5246c1][responses][field0] checkbox
surveysResponses[fc802a09-b71e-40bd-a979-34722f5246c1][responses][field0] checkbox
surveysResponses[fc802a09-b71e-40bd-a979-34722f5246c1][responses][field0] checkbox
surveysResponses[fc802a09-b71e-40bd-a979-34722f5246c1][res

In [4]:
dict = {}

for input_field in form_inputs:
    if input_field.get('type') == 'text':
        dict[input_field.get('name')] = input_field.get('value')

print(dict)
dict["name"]= "Wandrille"

{'name': None, 'phone': None, 'location': None, 'org': None}


In [5]:
DynamicFormClass = type(
    'DynamicForm',  # Class name
    (object,),      # Base classes
    dict     # Dictionary of attributes
)

# Instantiate the class
form_instance = DynamicFormClass()

In [6]:
form_instance.name

'Wandrille'

In [7]:
from langchain_community.llms import Ollama
from langchain_core.prompts import SystemMessagePromptTemplate
from langchain_core.prompts import HumanMessagePromptTemplate
from langchain_core.prompts import PromptTemplate

In [36]:
def load_llama3() -> Ollama:
    """
    Load the LLAMA3 model from OLLAMA.
    """

    ollama_client = Ollama(base_url="http://localhost:11434", model="llama3")

    system_message = ("""
        You are an expert AI data extraction assistant specialized in processing text-based information. 
        Your primary objective is to accurately analyze input text and extract structured information.

        Key Guidelines:
        - Extract information precisely from the given text
        - Be concise and accurate
        - Use null for missing information
        - Ensure output is valid, parseable JSON
        - Do not invent or assume information not present in the text""")

    SystemMessagePromptTemplate(
        prompt=PromptTemplate(
            input_variables=[],  # No input variables for the system message
            template=system_message,
        )
    )
    HumanMessagePromptTemplate(
        prompt=PromptTemplate(input_variables=["input"], template="{input}")
    )

    return ollama_client

In [37]:
from typing import Dict


def generate_answer_ollama(
    input_text: str,
    dict: Dict[str, str],
) -> str:
    """
    Generate a summary for a given text using the OLLAMA model.
    """

    ollama_model = load_llama3()

    aux_prompt = ""
    for key, value in dict.items():
        aux_prompt += f'{key}: <extracted {key} or null>,'
    prompt = f'''
    Task: Extract information from the following text and format the response EXACTLY as specified.

    Input: {input_text}

    STRICT OUTPUT REQUIREMENTS:
    1. You MUST respond ONLY in the following JSON format:
    {{{aux_prompt}}}

    CRITICAL INSTRUCTIONS:
    - If ANY information is missing, use null
    - Do NOT add ANY additional text or explanation
    - The output MUST be a valid JSON that can be parsed directly
    - Be precise in extraction
    - Only use information explicitly mentioned in the text

    EXAMPLE FORMAT:
    {{{aux_prompt}}}

    Respond ONLY with the JSON, exactly matching this structure.'''
    
    response = ollama_model(prompt)

    return response

In [38]:
response = generate_answer_ollama("I am Wandrille, a data scientist. I am very motivated to work with you.", dict)

In [39]:
response

'{ \n  "name": "Wandrille", \n  "phone": null, \n  "location": null, \n  "org": "data scientist" \n}'

In [40]:
import json

In [41]:
data = json.loads(response)

In [42]:
data

{'name': 'Wandrille', 'phone': None, 'location': None, 'org': 'data scientist'}